# Extraction de mots-clés à partir de descriptions
Ce notebook permet de charger un ensemble de thésaurus multilingues et d'extraire automatiquement des mots-clés à partir de descriptions en français ou anglais. 

## Préparation de l'environnement.

In [31]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from pathlib import Path
import langid


### Chargement du modèle de langage. 

In [ ]:
#model = SentenceTransformer("distiluse-base-multilingual-cased-v1") # MODEL 1
#model = SentenceTransformer("sentence-transformers/LaBSE") # MODEL 2
#model = SentenceTransformer("multi-qa-mpnet-base-cos-v1") #model 3 à essayer
model =  SentenceTransformer("dangvantuan/sentence-camembert-base") #model fr


### Chargement, encodage et stockage des thésaurus dans un dictionnaire. 

In [33]:
thesaurus_dir = Path(f"/Users/labo.adm/Documents/rdg/data/thesaurus/processed")  
thesaurus_files = list(thesaurus_dir.glob("*.csv"))

if not thesaurus_files:
    print(" Aucun fichier CSV trouvé dans", thesaurus_dir.resolve())
else:
    print(f" {len(thesaurus_files)} fichiers trouvés :", [f.name for f in thesaurus_files])

def load_thesaurus_and_encode(thesaurus_file):
    df = pd.read_csv(thesaurus_file, encoding="utf-8")
    if not {"fr", "en"}.issubset(df.columns):
        raise ValueError(f"Le fichier {thesaurus_file.name} ne contient pas les colonnes 'fr' et 'en'")
    
    df = df[["fr", "en"]].dropna()
    data = {}
    for lang in ["fr", "en"]:
        terms = df[lang].tolist()
        embeddings = model.encode(terms, convert_to_tensor=True)
        data[lang] = {
            "candidates": terms,
            "embeddings": embeddings
        }
    return data

thesaurus_data = {}
for file in thesaurus_files:
    try:
        name = file.stem
        thesaurus_data[name] = load_thesaurus_and_encode(file)
    except Exception as e:
        print(f" Erreur avec {file.name} : {e}")

print(f" Thésaurus chargés : {list(thesaurus_data.keys())}")


 4 fichiers trouvés : ['agrovoc_terms.csv', 'elsst_terms.csv', 'inrae_terms.csv', 'unesco_terms.csv']
 Thésaurus chargés : ['agrovoc_terms', 'elsst_terms', 'inrae_terms', 'unesco_terms']


On sauvegarde le dictionnaire créé avec les embeddings des thésaurus pour pouvoir le recharger et ne pas refaire l'étape précédente qui prend beaucoup de temps

In [34]:
# save thesaurus_data to a file
import pickle
thesaurus_data_file = Path("thesaurus_data.pkl")
with open(thesaurus_data_file, "wb") as f:
    pickle.dump(thesaurus_data, f)
print(f" Thésaurus sauvegardé dans {thesaurus_data_file.resolve()}")

 Thésaurus sauvegardé dans C:\Users\labo.adm\Documents\rdg\notebooks\thesaurus_data.pkl


On charge le dictionnaire précédemment enregistré. 

In [35]:
#load thesaurus_data from a file
with open(thesaurus_data_file, "rb") as f:
    thesaurus_data = pickle.load(f)

## Fonction qui prédit les mots-clés.

In [ ]:
def predict_keywords(doc, lang, thesaurus_data, n=5, threshold=0.25):
    if lang not in ["fr", "en"]:
        lang = "fr"  
    doc_embedding = model.encode(doc, convert_to_tensor=True)

    results_by_thesaurus = {}
    all_keywords = []

    for thesaurus_name, data in thesaurus_data.items():
        if lang not in data:
            continue

        candidates = data[lang]["candidates"]
        embeddings = data[lang]["embeddings"]
        scores = util.cos_sim(doc_embedding, embeddings)[0]

        thesaurus_results = []
        for kw, score in zip(candidates, scores):
            score_val = float(score)
            if score_val >= threshold:
                thesaurus_results.append((kw, score_val))
                all_keywords.append((kw, score_val, thesaurus_name))

        thesaurus_results = sorted(thesaurus_results, key=lambda x: x[1], reverse=True)[:n]
        results_by_thesaurus[thesaurus_name] = thesaurus_results

    top_all = sorted(all_keywords, key=lambda x: x[1], reverse=True)[:n]
    return results_by_thesaurus, top_all


## Description et détection de la langue.
Dans ce bloc on insère la description à partir de laquelle on veut extraire les mots-clés. 
Attention de bien l'insérer entre [""" """], cela évite des erreurs avec les guillemets présents dans le texte. 


In [37]:
descriptions = [
    """ 
Les données présentées ont été produites dans le contexte de l'étude des jeux sérieux, et plus particulièrement de la définition de la relation entre éléments pédagogiques et éléments ludiques établie par les modèles de jeux sérieux de la littérature. Les 4 tableaux composant ce jeu présentent des valeurs exprimant la représentativité d'un modèle (en colonnes) par rapport aux catégories d'un thésaurus de référence (en lignes). Pour chaque modèle, 2 valeurs sont calculées par thésaurus : une compensatoire (additive) et une semi-compensatoire (multiplicative). Les 2 thésauri utilisés ici sont éduthès (https://eduthes.cdc.qc.ca/vocab/index.php) et le Thésaurus Européen d'Éducation (https://vocabularyserver.com/tee/fr/). Pour chaque tableau, plusieurs données sont précisées par rapport aux valeurs (intervalle, quartiles, médiane, …). French (2025-05-07) """
]
languages = [langid.classify(text)[0] for text in descriptions]


### On applique la fonction à la description. 


In [38]:
for description, lang in zip(descriptions, languages):
    print(f"\n Description : {description}")
    print(f" Langue détectée : {lang}")

    results_struct, top_all = predict_keywords(description, lang, thesaurus_data)

    print("\n Voici les résultats par thésaurus :")
    for thesaurus_name, keywords in results_struct.items():
        keywords_str = " ; ".join(f"{kw} ({score:.3f})" for kw, score in keywords)
        print(f"  - {thesaurus_name}: {keywords_str}")

    print("\n Voici le Top 5 des mots-clés :")
    for kw, score, thesaurus in top_all:
        print(f"  {kw} ({score:.3f}) [{thesaurus}]")




 Description :  
Les données présentées ont été produites dans le contexte de l'étude des jeux sérieux, et plus particulièrement de la définition de la relation entre éléments pédagogiques et éléments ludiques établie par les modèles de jeux sérieux de la littérature. Les 4 tableaux composant ce jeu présentent des valeurs exprimant la représentativité d'un modèle (en colonnes) par rapport aux catégories d'un thésaurus de référence (en lignes). Pour chaque modèle, 2 valeurs sont calculées par thésaurus : une compensatoire (additive) et une semi-compensatoire (multiplicative). Les 2 thésauri utilisés ici sont éduthès (https://eduthes.cdc.qc.ca/vocab/index.php) et le Thésaurus Européen d'Éducation (https://vocabularyserver.com/tee/fr/). Pour chaque tableau, plusieurs données sont précisées par rapport aux valeurs (intervalle, quartiles, médiane, …). French (2025-05-07) 
 Langue détectée : fr

 Voici les résultats par thésaurus :
  - agrovoc_terms: Théorie axiomatique des ensembles (0.329

## Fonction pour avoir la traduction des mots-clés. 

In [39]:
def get_translations_by_thesaurus_index(thesaurus_data, keywords_by_thesaurus, lang):
    def normalize(s):
        return s.strip().lower()

    other_lang = "fr" if lang == "en" else "en"
    translations_by_thesaurus = {}

    for thesaurus_name, detected_keywords in keywords_by_thesaurus.items():
        if thesaurus_name not in thesaurus_data:
            continue

        data = thesaurus_data[thesaurus_name]
        if lang not in data or other_lang not in data:
            continue

        source_list = data[lang]["candidates"]
        target_list = data[other_lang]["candidates"]

        # Création d'une map normalisée : terme normalisé → index
        normalized_index_map = {
            normalize(term): i for i, term in enumerate(source_list)
        }

        translations = []
        for kw in detected_keywords:
            norm_kw = normalize(kw)
            if norm_kw in normalized_index_map:
                idx = normalized_index_map[norm_kw]
                if idx < len(target_list):
                    translations.append(target_list[idx])
                else:
                    translations.append("(non traduit)")
            else:
                translations.append("(non traduit)")

        translations_by_thesaurus[thesaurus_name] = translations

    return translations_by_thesaurus



### On applique la fonction précédente et on affiche les résultats.

In [ ]:
# Affichage des mots-clés dans l'autre langue
for description, lang in zip(descriptions, languages):
    print(f"\nDescription : {description}")
    print(f"Langue détectée : {lang}")

    results_struct, top_all = predict_keywords(description, lang, thesaurus_data)


keywords_by_thesaurus = {
    th_name: [kw for kw, *_ in kws]
    for th_name, kws in results_struct.items()
}

translations_by_thesaurus = get_translations_by_thesaurus_index(
    thesaurus_data,
    keywords_by_thesaurus,
    lang
)

print(f"\nVoici les mots-clés et leur traduction ({'en' if lang == 'fr' else 'fr'}) :")
for thesaurus_name, translations in translations_by_thesaurus.items():
    print(f"  - {thesaurus_name}:")
    for original_kw, translated_kw in zip(keywords_by_thesaurus[thesaurus_name], translations):
        print(f"      {original_kw} → {translated_kw}")




Description :  
Les données présentées ont été produites dans le contexte de l'étude des jeux sérieux, et plus particulièrement de la définition de la relation entre éléments pédagogiques et éléments ludiques établie par les modèles de jeux sérieux de la littérature. Les 4 tableaux composant ce jeu présentent des valeurs exprimant la représentativité d'un modèle (en colonnes) par rapport aux catégories d'un thésaurus de référence (en lignes). Pour chaque modèle, 2 valeurs sont calculées par thésaurus : une compensatoire (additive) et une semi-compensatoire (multiplicative). Les 2 thésauri utilisés ici sont éduthès (https://eduthes.cdc.qc.ca/vocab/index.php) et le Thésaurus Européen d'Éducation (https://vocabularyserver.com/tee/fr/). Pour chaque tableau, plusieurs données sont précisées par rapport aux valeurs (intervalle, quartiles, médiane, …). French (2025-05-07) 
Langue détectée : fr

Voici les mots-clés et leur traduction (en) :
  - agrovoc_terms:
      Théorie axiomatique des ens